# Structured Output Tutorial

## Overview

This tutorial delves into tehcnique of function calling in language models to achieve structured outputs from any LLM.

## Motivation

In the rapidly evolving field of artificial intelligence, the ability to generate structured outputs from language models is becoming increasingly important. Structured outputs allow for more precise and interpretable results, which are essential for applications ranging from data extraction to automated decision-making.

## Key Components

1. Understanding how to invoke specific functions within a language model to generate structured outputs.
2. Methods for ensuring that the generated outputs conform to the expected structure and format.
3. Practical examples of integrating structured output generation with popular frameworks like LangChain and OpenRouter.

## Setup

First, let's import the necessary libraries and set up our environment.

In [1]:
import json
from datetime import datetime
from os import getenv

from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel, Field

load_dotenv()

# Initialize the language model
llm = OpenAI(
    api_key=getenv("OPENROUTER_API_KEY"),
    base_url=getenv("OPENROUTER_BASE_URL"),
)

## Unstructured Output

In [2]:
# Define your prompt
prompt = "Can you tell me about Harry Potter's profile?"

# Make the LLM call
response = llm.chat.completions.create(
    model="azure/gpt-4o",
    messages=[{"role": "user", "content": prompt}],
)

# Print the result
print(response.choices[0].message.content)

Certainly! Harry Potter is the iconic protagonist of the **Harry Potter series** written by J.K. Rowling. He is a young wizard whose adventures have captivated readers and audiences around the world. Here is a detailed profile of Harry:

---

### **Personal Information:**
- **Full Name:** Harry James Potter  
- **Birthdate:** July 31, 1980
- **Blood Status:** Half-blood  
- **Parents:** James Potter (pure-blood) and Lily Potter (Muggle-born)
- **Godfather:** Sirius Black  
- **Spouse:** Ginny Weasley (married after the events of the series)  
- **Children:** James Sirius, Albus Severus, and Lily Luna Potter  

---

### **Physical Appearance:**
- **Hair:** Jet-black and perpetually messy  
- **Eyes:** Bright green, inherited from his mother  
- **Scar:** Lightning-shaped scar on his forehead from a curse by Lord Voldemort  
- **Height:** Average for his age  

---

### **Personality Traits:**
- Brave, loyal, and self-sacrificing  
- Quick to stand up for what is right  
- Strong sense o

## Structured Output

![Structured Output](https://storage.googleapis.com/tqtensor-sharing/structured_output.png)

In [3]:
class Person(BaseModel):
    name: str = Field(description="The person's full name")
    date_of_birth: datetime = Field(
        description="The person's date of birth in ISO 8601 format (YYYY-MM-DD)"
    )
    occupation: str = Field(description="The person's current job or profession")


def get_person_info() -> Person:
    """Get information about a person from LLM response."""

    return Person(name="", age=0, occupation="")


# Create the function description for the API
function_description = {
    "name": "get_person_info",
    "description": "Get information about a person from LLM response.",
    "parameters": Person.model_json_schema(),
}

# Make the LLM call
response = llm.chat.completions.create(
    model="azure/gpt-4o",
    messages=[{"role": "user", "content": prompt}],
    functions=[function_description],
    function_call={"name": "get_person_info"},
)

# Extract the function call arguments
function_call = response.choices[0].message.function_call.arguments
arguments = json.loads(function_call)

# Create a Person object from the arguments
person = Person(**arguments)

# Print the result
print(f"Name: {person.name}")
print(f"DoB: {person.date_of_birth}")
print(f"Occupation: {person.occupation}")

Name: Harry Potter
DoB: 1980-07-31 00:00:00
Occupation: Auror
